In [91]:
!pip python-terrier


2004.79s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
ERROR: unknown command "python-terrier"
Note: you may need to restart the kernel to use updated packages.


In [92]:
import pyterrier as pt
if not pt.started():
  pt.init()

# Disable SSL certificate verification
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [93]:
import pandas as pd
import os
import json

# Specify the directory path relative to the notebook location
data_dir = os.path.abspath("./../chocolate_crawler/Crawled/")

# List of JSON files
json_files = ['laderach.json', 'spruengli.json', 'maxchocolatier.json']

# Initialize an empty list to store data from all JSON files
all_data = []

# Load data from each JSON file
for json_file in json_files:
    json_path = os.path.join(data_dir, json_file)

    print(f"Loading data from {json_path}")

    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
            all_data.extend(data)
    except FileNotFoundError:
        print(f"File not found: {json_path}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in {json_path}: {e}")


# Create an index
idx = ['d' + str(i + 1) for i in range(len(all_data))]


# Extract relevant information
titles = [item.get('title', '') for item in all_data]
descriptions = [item.get('description', '') for item in all_data]
ingredients = [item.get('ingredients', '') for item in all_data]
allergens = [item.get('allergens', '') for item in all_data]
prices = [item.get('price', '') for item in all_data]

# Create a DataFrame
docs_df = pd.DataFrame(np.column_stack((idx, titles, descriptions, ingredients, allergens, prices)),
                       columns=['docno', 'title', 'description', 'ingredients', 'allergens', 'price'])

docs_df


Loading data from /Users/sapanatamang/Desktop/ir-choco/chocolate_crawler/Crawled/laderach.json
Loading data from /Users/sapanatamang/Desktop/ir-choco/chocolate_crawler/Crawled/spruengli.json
Loading data from /Users/sapanatamang/Desktop/ir-choco/chocolate_crawler/Crawled/maxchocolatier.json


,docno,title,description,ingredients,allergens,price
0,d1,FrischSchoggi Hazelnut Milk,This FrischSchoggi made of fine milk chocolate...,"31%, sugar, cocoa butter, whole powder, cocoa ...","May contain egg, gluten (incl. wheat), other n...",CHF 21.10
1,d2,FrischSchoggi Sticks Box,We hope your Christmas is as colourful and swe...,"sugar, cocoa butter, , whole powder, cocoa pas...","May contain egg, gluten (incl. wheat), other n...",CHF 21.10
2,d3,FrischSchoggi Sticks Christmasbox dark mini,We hope your Christmas is as colourful and swe...,"cocoa paste, sugar, cocoa butter, , , orange j...","May contain egg, gluten (incl. wheat), milk, o...",CHF 21.10
3,d4,FrischSchoggi Sticks Christmasbox mini,We hope your Christmas is as colourful and swe...,"sugar, cocoa butter, , whole powder, cocoa pas...","May contain egg, gluten (incl. wheat), other n...",CHF 21.10
4,d5,Christmas Pralines assorted 72pcs per box,A selection of 72 beautifully crafted Christma...,"sugar, cocoa butter, cocoa paste, whole powder...","May contain egg, other nuts.",CHF 21.10
...,...,...,...,...,...,...
675,d676,Box with 24 assorted winter truffles,A selection of 24 handmade truffles from our w...,"Sugar, cocoa nibs, whole cream , cocoa butter,...",,52.80
676,d677,Dragée bag Christmas almonds,Caramelized Californian almonds coated in Gran...,"Sugar, almonds 23%, cocoa butter, cocoa seeds,...",,12.85
677,d678,"Bear with Christmas tree ""Bolivia 45%""","Teddy bear made from Grand Cru chocolate ""Boli...","Sugar, cocoa butter, cocoa nibs, skimmed milk ...",,15.90
678,d679,"Bear with Christmas tree ""Madagascar 68%""",Teddy bear made from dark Grand Cru chocolate ...,"Cocoa seeds, sugar, cocoa butter, vanilla Mada...",,15.90


In [94]:
indexer = pt.DFIndexer("./index", overwrite=True)
index_ref = indexer.index(docs_df["description"], docs_df["docno"])
index_ref.toString()

'./index/data.properties'

In [95]:
!ls -lh index/

2010.68s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
total 3176
-rw-r--r--@ 1 sapanatamang  staff   653K Nov 30 22:03 chocolate_index.csv
-rw-r--r--@ 1 sapanatamang  staff   653K Nov 30 22:02 chocolate_index.txt
-rw-r--r--@ 1 sapanatamang  staff    25K Nov 30 22:44 data.direct.bf
-rw-r--r--@ 1 sapanatamang  staff    11K Nov 30 22:44 data.document.fsarrayfile
-rw-r--r--@ 1 sapanatamang  staff    20K Nov 30 22:44 data.inverted.bf
-rw-r--r--@ 1 sapanatamang  staff   157K Nov 30 22:44 data.lexicon.fsomapfile
-rw-r--r--@ 1 sapanatamang  staff   993B Nov 30 22:44 data.lexicon.fsomaphash
-rw-r--r--@ 1 sapanatamang  staff   7.3K Nov 30 22:44 data.lexicon.fsomapid
-rw-r--r--@ 1 sapanatamang  staff    11K Nov 30 22:44 data.meta-0.fsomapfile
-rw-r--r--@ 1 sapanatamang  staff   5.3K Nov 30 22:44 data.meta.idx
-rw-r--r--@ 1 sapanatamang  staff    14K Nov 30 22:44 data.meta.zdata
-rw-r--r--@ 1 sapanatamang  staff   4.1K Nov 30 22:44 data.properties


In [96]:
index = pt.IndexFactory.of(index_ref)

type(index)

jnius.reflect.org.terrier.structures.Index

In [97]:
print(index.getCollectionStatistics().toString())

Number of documents: 680
Number of terms: 1869
Number of postings: 30347
Number of fields: 0
Number of tokens: 37490
Field names: []
Positions:   false



In [98]:
for kv in index.getLexicon():
  print("%s  -> %s " % (kv.getKey(), kv.getValue().toString()  ))

1  -> term1741 Nt=6 TF=6 maxTF=1 @{0 0 0} 
100  -> term1187 Nt=2 TF=2 maxTF=1 @{0 7 4} 
1000g  -> term491 Nt=1 TF=1 maxTF=1 @{0 10 4} 
100g  -> term375 Nt=9 TF=9 maxTF=1 @{0 12 2} 
105g  -> term158 Nt=6 TF=6 maxTF=1 @{0 20 0} 
11  -> term602 Nt=1 TF=1 maxTF=1 @{0 23 2} 
110g  -> term198 Nt=5 TF=5 maxTF=1 @{0 25 2} 
12  -> term614 Nt=7 TF=7 maxTF=1 @{0 29 6} 
120g  -> term277 Nt=4 TF=4 maxTF=1 @{0 40 0} 
125g  -> term603 Nt=1 TF=1 maxTF=1 @{0 44 6} 
130g  -> term508 Nt=5 TF=5 maxTF=1 @{0 46 6} 
140g  -> term516 Nt=1 TF=1 maxTF=1 @{0 52 0} 
144  -> term111 Nt=1 TF=1 maxTF=1 @{0 53 6} 
15  -> term1079 Nt=2 TF=2 maxTF=1 @{0 54 6} 
150g  -> term139 Nt=1 TF=1 maxTF=1 @{0 57 0} 
16  -> term310 Nt=15 TF=15 maxTF=1 @{0 58 0} 
1680g  -> term110 Nt=1 TF=1 maxTF=1 @{0 78 2} 
170g  -> term611 Nt=1 TF=1 maxTF=1 @{0 79 2} 
18  -> term115 Nt=4 TF=4 maxTF=1 @{0 81 2} 
180  -> term763 Nt=241 TF=241 maxTF=1 @{0 86 2} 
180g  -> term309 Nt=2 TF=2 maxTF=1 @{0 160 2} 
1811  -> term987 Nt=1 TF=1 maxTF=1 @{0 1

In [99]:
# print(index.getLexicon()["document"].toString())
# print(index.getLexicon()["first"].toString())
# print(index.getLexicon()["topic"].toString())
# print(index.getLexicon()["unknown"].toString())

for kv in index.getLexicon() :
  print(kv.getKey())
  print(index.getLexicon()[kv.getKey()].toString())
  print('**************************************************')

1
term1741 Nt=6 TF=6 maxTF=1 @{0 0 0}
**************************************************
100
term1187 Nt=2 TF=2 maxTF=1 @{0 7 4}
**************************************************
1000g
term491 Nt=1 TF=1 maxTF=1 @{0 10 4}
**************************************************
100g
term375 Nt=9 TF=9 maxTF=1 @{0 12 2}
**************************************************
105g
term158 Nt=6 TF=6 maxTF=1 @{0 20 0}
**************************************************
11
term602 Nt=1 TF=1 maxTF=1 @{0 23 2}
**************************************************
110g
term198 Nt=5 TF=5 maxTF=1 @{0 25 2}
**************************************************
12
term614 Nt=7 TF=7 maxTF=1 @{0 29 6}
**************************************************
120g
term277 Nt=4 TF=4 maxTF=1 @{0 40 0}
**************************************************
125g
term603 Nt=1 TF=1 maxTF=1 @{0 44 6}
**************************************************
130g
term508 Nt=5 TF=5 maxTF=1 @{0 46 6}
**********************************************

In [100]:
word_ = 'chocolate'
pointer = index.getLexicon()[word_]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

KeyError: 

In [ ]:
br = pt.BatchRetrieve(index, wmodel="BM25") #Alternative Models: "TF_IDF", "BM25"
br.search("truffle")

In [ ]:
br.search("christmas")